In [0]:
pip install nltk

Python interpreter will be restarted.
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
  Using cached regex-2022.1.18-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5027cd62-fefb-4907-b0f4-f6a2508581e2/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


In [0]:
plotSummaries = sc.textFile('/FileStore/tables/plot_summaries.txt')

In [0]:
moviesMetadata = spark.read.csv("/FileStore/tables/movie_metadata.tsv", sep=r'\t', header=False) # here we get dataframe
movies = moviesMetadata.rdd # converts dataframe to rdd
movieNames = movies.map(lambda row : (row._c0, row._c2)) # maps over rdd
movies = movieNames.collectAsMap()

In [0]:
summaries = plotSummaries.map(lambda line : line.split('\t')).map(lambda x : (x[0], x[1])) # get key-value tuple pair using maps

In [0]:
summaries.take(10)

Out[15]: [('23890098',
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."),
 ('31186339',
  'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Out[4]: True

In [0]:
import string
from nltk.corpus import stopwords
# return words which are not in the list of stopwords(which is list of stopwords and punctuated tokens)
def removeInvalidWords(words):
    stopWords = list(stopwords.words('english'))
    puncs = string.punctuation+' '
    ansList = []
    for word in words:
        word = word.strip(puncs)
        if len(word) == 0:
            continue
        if word not in stopWords:
            ansList.append(word)
    return ansList

In [0]:
# converts summary to list of words and removes stopwords from them
def removeStopWords(summary):
    words = summary.lower().split(" ")
    newWords = removeInvalidWords(words)
    return newWords

In [0]:
# maps through summaries and returns list of summary words
summaries = summaries.map(lambda x : (x[0], removeStopWords(x[1])))

In [0]:
N = summaries.count() # total count of all movie documents

In [0]:
# calculating tij which is count of ith word in a jth movie document
summary = summaries.flatMapValues(lambda x : x)
tij = summary.map(lambda t : ((t[0], t[1]), 1)).reduceByKey(lambda x, y : x+y)
tij.take(10)

Out[33]: [(('1952976', 'refused'), 1),
 (('2462689', 'publisher'), 1),
 (('1760737', 'balances'), 1),
 (('5414895', 'felt'), 1),
 (('5414895', 'sister'), 1),
 (('1838882', 'order'), 1),
 (('17711304', 'washington'), 1),
 (('17711304', 'order'), 3),
 (('529276', 'martin'), 1),
 (('529276', 'stop'), 1)]

In [0]:
# calculating count of a ith word in all the movie documents
ni = summary.distinct().map(lambda t : (t[1], t[0])).reduceByKey(lambda x, y : x+" "+y).map(lambda t : (t[0], len(t[1].split(" "))))
ni = ni.collectAsMap()
ni

Out[34]: {'played': 1254,
 'working': 3392,
 'custody': 660,
 'entrance': 408,
 'act': 1836,
 'counselor': 125,
 'call': 2867,
 'hearings': 27,
 'live': 3852,
 'prepare': 765,
 'scientists': 362,
 'tells': 9530,
 'try': 3807,
 'differences': 362,
 'burcu': 1,
 'footing': 31,
 'making': 3494,
 'dark': 1227,
 'conditions': 373,
 'singing': 1056,
 'zeke': 24,
 'rich--dirty': 1,
 'shinkengers': 3,
 'so-mi': 1,
 'private': 1288,
 'true': 2592,
 'accepts': 1952,
 'rizolli': 1,
 'owner': 2100,
 'prepared': 395,
 'got': 1568,
 'hostages': 180,
 'companionship': 66,
 'major': 1302,
 'star.there': 1,
 'feral': 37,
 'end': 6620,
 'something': 2458,
 'several': 4336,
 "hannaford's": 1,
 'ebenezer': 16,
 'leadership': 223,
 'utopian': 20,
 'starts': 4875,
 "world's": 260,
 'europe': 538,
 'decides': 7357,
 'chasm': 39,
 'york': 2418,
 'apparently': 1351,
 'painted': 160,
 'elinu': 1,
 'savanah': 1,
 'anny': 2,
 'leading': 1872,
 'citizens': 398,
 'charges': 734,
 'interest': 1540,
 'school': 4396,


In [0]:
# calculating and saving the results of tf-idf for the dataset
import math
tfIdf = tij
tfIdf = tfIdf.map(lambda x : (x[0], float(x[1]*float(math.log(N/ni[x[0][1]]))))).sortBy(lambda x : -x[1])
tfIdf = tfIdf.collectAsMap()

In [0]:
tfIdf

Out[36]: {('21768047', 'keroro'): 991.503712647511,
 ('6283620', 'hyun'): 949.5408600838645,
 ('6283620', 'soo'): 939.2580213502184,
 ('22175264', 'timmy'): 837.9563369335178,
 ('15465392', 'lucian'): 822.9354120726825,
 ('7548602', 'marcellus'): 818.4367606898276,
 ('4457831', 'morgaine'): 809.6039991199793,
 ('6283620', 'eun'): 742.9310033533852,
 ('28443267', 'luk'): 711.5791405531826,
 ('4969625', 'orry'): 703.0771571305083,
 ('3926180', 'franklin'): 692.1518727474786,
 ('11467890', 'sabata'): 687.893177540087,
 ('11584626', 'hyeon-su'): 671.119104533667,
 ('15769706', 'vamsi'): 669.2002252019616,
 ('2650227', 'janie'): 665.4244736118126,
 ('10619079', 'genocyber'): 660.4664203347199,
 ('21673015', 'ged'): 651.0703908640374,
 ('25092646', 'obie'): 637.4103691767776,
 ('35581515', 'hushpuppy'): 617.8556835389315,
 ('2065944', 'shogo'): 617.4912951400033,
 ('3257858', 'gidget'): 614.9407474828836,
 ('23325348', 'cherie'): 601.0220151998877,
 ('20887118', 'cavil'): 596.5503151410373,


In [0]:
movies

Out[37]: {'975900': 'Ghosts of Mars',
 '3196793': 'Getting Away with Murder: The JonBenét Ramsey Mystery',
 '28463795': 'Brun bitter',
 '9363483': 'White Of The Eye',
 '261236': 'A Woman in Flames',
 '13696889': 'The Gangsters',
 '18998739': "The Sorcerer's Apprentice",
 '10408933': "Alexander's Ragtime Band",
 '9997961': 'Contigo y aquí',
 '2345652': 'City of the Dead',
 '175026': 'Sarah and Son',
 '24229100': 'Lady Snowblood 2: Love Song of Vengeance',
 '6631279': 'Little city',
 '171005': 'Henry V',
 '18296435': 'Aaah Belinda',
 '11250635': 'The Mechanical Monsters',
 '30388930': '1919',
 '77856': 'Mary Poppins',
 '32456683': 'Die Fahne von Kriwoj Rog',
 '33420460': 'Keep the Change',
 '175024': "The Devil's Holiday",
 '612710': 'New Rose Hotel',
 '21926710': 'White on Rice',
 '33427105': 'Freddy and the Song of the South Pacific',
 '31983669': 'Road to Life',
 '17715326': 'Camera Thrills',
 '22087420': 'Ferdinando I, re di Napoli',
 '20604092': 'Anbu Thozhi',
 '21344842': 'Middle A

In [0]:
def searchEngine_single(query, topHitsCount=10):
#     output = tfIdf.filter(lambda x : x[0][1] == query).take(10)  
    output = []
    for values in tfIdf.keys():
        if values[1] == query:
            output.append(values[0])
    ans = []
    for val in range(min(len(output), topHitsCount)):
        ans.append(movies[output[val]])
    print("Results : "+str(ans))

In [0]:
from scipy import spatial
def searchEngine_multiple(query, topHitsCount=10):
    qcount = {}
    for q in query:
        if q not in qcount.keys():
            qcount[q] = 1
        else:
            qcount[q]+=1
    words = []
    qVector = []
    for key, value in qcount.items():
        words.append(key)
        qVector.append(value)

    movieId = list(movies.keys())
    cosine = {}
    for doc in movieId:
        dVector = []
        for word in words:
            t = (doc, word)
            if t in tfIdf:
                dVector.append(tfIdf[t])
            else:
                dVector.append(0)
        s = set(dVector)
        if len(s) == 1 and 0 in s:
            continue
        result = 1 - spatial.distance.cosine(qVector, dVector)
        cosine[doc] = result
    
    sortedHits = list(sorted(cosine.items(), key=lambda item: -item[1])) # list maintains the order of sort
    sortedHits = sortedHits[0:min(len(sortedHits), topHitsCount)]
    ans=[]
    for k in range(len(sortedHits)):# getting movie names for the resultant movie ids
        temp_key = sortedHits[k][0] # movie id
        #sortedHits[k]= (movies[temp_key], sortedHits[k][1])
        ans.append(movies[temp_key])
    print("Results : "+str(ans))

In [0]:
query_file = spark.read.csv("/FileStore/tables/queries-1.csv") # all search queries stored at first row of the csv file with delimiter','
queries=[]
for i in query_file.collect()[0]:
    queries.append([removeStopWords(i.strip().lower()),i])
for query in queries:
    print("\n")
    print('For Query "'+ query[1]+'"')
    if len(query[0])==0:
        print("Result: Query is invalid")
        continue
    
    if len(query[0])>1:
        searchEngine_multiple(query[0])
    else:
        searchEngine_single(query[0][0])



For Query "horror"
Results : ['Garo: Red Requiem', 'The Last Horror Film', 'The Pagemaster', 'The Pit and the Pendulum', 'Microwave Massacre', 'Fright Night', 'Kiba Gaiden', 'Lightning Bug', 'The Grudge 3', 'Memphis Belle']


For Query "funny"
Results : ["Garfield's Fun Fest", 'The Last Circus', 'Bheja fry', 'Ullathai Allitha', 'Funny Man', 'Heart Like a Wheel', 'Main Sunder Hoon', '14 Carrot Rabbit', 'Ernest Goes to Jail', 'War Dogs']


For Query "The heart"
Results : ['Care Bears Movie II: A New Generation', 'The Care Bears Adventure in Wonderland', 'Crank: High Voltage', 'Captain Sindbad', "Pirates of the Caribbean: At World's End", 'Snow White', 'Dragonheart: A New Beginning', 'Frankenstein Conquers the World', 'Bordello of Blood', '21 Grams']


For Query "Funny Movies with Action scenes"
Results : ['The Muppets: A Celebration of 30 Years', 'Three the Hard Way', 'Megazone 23', 'Tony', "Fellini: I'm a Born Liar", 'Ullathai Allitha', 'Nell', 'Imtihaan', 'Om Shanti Om', 'My Name is 